# Create temp tables

Use this script in a notebook to create temp collections for a projet :
- events
- sessions
- flattened_tasks

This is helpful to use with the postgresql export. 

In [ ]:
from app.db.mongo import connect_and_init_db, get_mongo_db
from loguru import logger

await connect_and_init_db()

mongo_db = await get_mongo_db()

# Change the project id here
project_id = ...


# Delete the collections if they exist
logger.info("Deleting collections")
await mongo_db[f"events_{project_id}"].drop()
await mongo_db[f"sessions_{project_id}"].drop()
await mongo_db[f"flattened_tasks_{project_id}"].drop()

# Create a new collection with only the events of project

command = [
    {
        "$match": {
            "project_id": project_id
    },
    },
    {
        "$out": f"events_{project_id}"
    }
]

logger.info("Creating events collection")
await mongo_db["events"].aggregate(command).to_list(None)

# Same with sessions

command = [
    {
        "$match": {
            "project_id": project_id
        },
    },
    {
        "$out": f"sessions_{project_id}"
    }
]

logger.info("Creating sessions collection")
await mongo_db["sessions"].aggregate(command).to_list(None)

# Create indexes on the new collections: task_id

logger.info("Creating indexes on events collection")
await mongo_db[f"events_{project_id}"].create_index("task_id", unique=False, background=False)

logger.info("Creating indexes on sessions collection")
await mongo_db[f"sessions_{project_id}"].create_index("task_id",unique=False, background=False)


# Run the command
command = [
  {
    "$match": {
      "project_id": project_id,
    },
  },
  { "$sort": { "created_at": 1 } },
  {
    "$lookup": {
      "from": f"sessions_{project_id}",
      "localField": "id",
      "foreignField": "task_id",
      "as": "sessions",
    },
  },
  {
    "$unwind": {
      "path": "$sessions",
      "preserveNullAndEmptyArrays": True,
    },
  },
  {
    "$lookup": {
      "from": f"events_{project_id}",
      "localField": "id",
      "foreignField": "task_id",
      "as": "events",
    },
  },
  {
    "$addFields": {
      "events": {
        "$filter": {
          "input": "$events",
          "as": "event",
          "cond": {
            "$and": [
              { "$ne": ["$$event.removed", True] },
              {
                "$or": [
                  {
                    "$and": [
                      {
                        "$eq": [
                          "$$event.event_definition.is_last_task",
                          True,
                        ],
                      },
                      {
                        "$eq": [
                          "$is_last_task",
                          True,
                        ],
                      },
                    ],
                  },
                  {
                    "$not": [
                      "$$event.event_definition.is_last_task",
                    ],
                  },
                ],
              },
            ],
          },
        },
      },
    },
  },
  {
    "$set": {
      "events": {
        "$reduce": {
          "input": "$events",
          "initialValue": [],
          "in": {
            "$concatArrays": [
              "$$value",
              {
                "$cond": [
                  {
                    "$in": [
                      "$$this.event_definition.id",
                      "$$value.event_definition.id",
                    ],
                  },
                  [],
                  ["$$this"],
                ],
              },
            ],
          },
        },
      },
    },
  },
  {
    "$unwind": {
      "path": "$events",
      "preserveNullAndEmptyArrays": True,
    },
  },
  {
    "$project": {
        "_id": 0,
      "task_id": "$id",
      "task_input": "$input",
      "task_output": "$output",
      "task_metadata": "$metadata",
      "task_eval": "$flag",
      "task_eval_source": "$last_eval.source",
      "task_eval_at": "$last_eval.created_at",
      "task_created_at": "$created_at",
      "session_id": "$session_id",
      "task_position": "$task_position",
      "session_length": "$sessions.length",
      "event_name": "$events.event_name",
      "event_created_at": "$events.created_at",
      "event_confirmed": "$events.confirmed",
      "event_score_range_value":
        "$events.score_range.value",
      "event_score_range_min":
        "$events.score_range.min",
      "event_score_range_max":
        "$events.score_range.max",
      "event_score_range_score_type":
        "$events.score_range.score_type",
      "event_score_range_label":
        "$events.score_range.label",
      "event_source": "$events.source",
      "event_categories":
        "$events.event_definition.score_range_settings.categories",
    },
  },
  # Merge into a new collection
    {
        "$out": f"flattened_tasks_{project_id}"
    },
]

logger.info("Creating flattened_tasks collection")
await mongo_db["tasks"].aggregate(command).to_list(None)